# this notebook is for the aggregation of all the clusters across subjects
The idea, then is to create a data frame containing each clusted unit, the snr, av. fr, isi viol, unique id within for the animal, an unique id within the cross session match. Other identifiers, session, tetrode, depth. 

In [283]:
%matplotlib inline

from pathlib import Path
import time
import traceback
from importlib import reload

import numpy as np
import scipy.stats as stats
import pandas as pd
pd.set_option('display.max_rows', 50)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', palette='muted')
from matplotlib_venn import venn2, venn3

from TreeMazeAnalyses2.Analyses import subject_info as si
from TreeMazeAnalyses2.Analyses import cluster_match_functions as cmf

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
subjects = ['Li', 'Ne', 'Cl', 'Al', 'Ca', 'Mi']

## Update Clusters if changes in phy

In [ ]:
%%time
if 0:  ## doesn't run by default
    si = reload(si)
    overwrite = False
    for subject in subjects:
        subject_info = si.SubjectInfo(subject, overwrite=overwrite)
        subject_info.update_clusters()   ## updates cluster snr/ fr isi info based on phy changes.

## Update cluster matches

In [ ]:
%%time
si = reload(si)
if 0:
    overwrite = True
    for subject in subjects:
        try:
            subject_info = si.SubjectInfo(subject, overwrite=overwrite)
            subject_info.get_cluster_dists(overwrite=overwrite)
            subject_info.match_clusters(overwrite=overwrite)
            subject_info.match_clusters(overwrite=overwrite, require_subsets=False)
        except:
            print(f"{subject} failed.")

## Update tables

In [ ]:
%%time
si = reload(si)
if 0:
    overwrite = True
    for subject in subjects:
        _ = subject_info.get_units_table(overwrite=overwrite)
        

## Aggregate Tables

In [402]:
si = reload(si)

# initialized counters
unit_cnt=0
unit_con_match_cnt = 0
unit_lib_match_cnt = 0

# iterate through subjects
all_units_table = pd.DataFrame()
for subject in subjects:
    subject_info = si.SubjectInfo(subject)
    
    tbl = subject_info.get_units_table(overwrite=True)
    tbl['cl_id'] = tbl.subject_cl_id + unit_cnt 
    tbl['cl_match_con_id'] =  tbl.subject_cl_match_con_id + unit_con_match_cnt 
    tbl['cl_match_lib_id'] =  tbl.subject_cl_match_lib_id + unit_lib_match_cnt
    
    unit_cnt = np.nanmax(tbl.cl_id)+1
    unit_con_match_cnt = np.nanmax(tbl.cl_match_con_id)+1
    unit_lib_match_cnt = np.nanmax(tbl.cl_match_lib_id)+1
    
    all_units_table = pd.concat((all_units_table, tbl))

all_units_table =  all_units_table.reset_index(drop=True)

# fix error on session naming for session Li_OF_080718, this is a T3g session- bug discovered on 3/25/18
all_units_table['task2'] = all_units_table.task
all_units_table.loc[all_units_table.session=="Li_OF_080718", 'task'] = 'T3g'
all_units_table.loc[all_units_table.session=="Li_OF_080718", 'task2'] = 'T3'
all_units_table.loc[list(map(isT3, all_units_table.task)),'task2'] = 'T3'

# find multi task unit matches
def isT3(a):
    return 'T3' in a

all_units_table['match_lib_multi_task_id'] = -1
unit_matches = np.arange(np.nanmax(all_units_table.cl_match_lib_id)+1)
cnt=0
for cm  in unit_matches:
    tasks_in_cm = all_units_table[all_units_table.cl_match_lib_id==cm].task2.values
    if ('OF' in tasks_in_cm) & ('T3' in tasks_in_cm):
        all_units_table.loc[all_units_table.cl_match_lib_id==cm, 'match_lib_multi_task_id'] = cnt
        cnt+=1
                                  
all_units_table['match_con_multi_task_id'] = -1
unit_matches = np.arange(np.nanmax(all_units_table.cl_match_con_id)+1)
cnt=0
for cm  in unit_matches:
    tasks_in_cm = all_units_table[all_units_table.cl_match_con_id==cm].task2.values
    if ('OF' in tasks_in_cm) & ('T3' in tasks_in_cm):
        all_units_table.loc[all_units_table.cl_match_con_id==cm, 'match_con_multi_task_id'] = cnt
        cnt+=1
            

In [413]:
a = session_info.get_track_data()


0.9198250728862973

## Save to GDrive

In [403]:
save_path_gd = Path("/home/alexgonzalez/google-drive/TreeMazeProject/Results_Summary/")
save_path_local = Path("/mnt/Data_HD2T/SummaryData/")

fn = "all_units_table.csv"

all_units_table.to_csv(save_path_gd/fn)
all_units_table.to_csv(save_path_local/fn)

In [392]:
all_units_table

,subject_cl_id,subject,session,task,date,subsession,tt,depth,unique_cl_name,session_cl_id,...,subject_cl_match_lib_id,snr,fr,isi_viol_rate,cl_id,cl_match_con_id,cl_match_lib_id,task2,match_lib_multi_task_id,match_con_multi_task_id
0,0,Li,Li_T3g_052818,T3g,052818,0000,2,16.5,Li_T3g_052818-tt2_d16.5_cl11,11,...,NaN,NaN,2.84,0.2,0,NaN,NaN,T3,-1,-1
1,1,Li,Li_T3g_052818,T3g,052818,0000,2,16.5,Li_T3g_052818-tt2_d16.5_cl14,14,...,NaN,NaN,2.69,0.05,1,NaN,NaN,T3,-1,-1
2,2,Li,Li_OF_052818,OF,052818,0000,7,17.25,Li_OF_052818-tt7_d17.25_cl8,8,...,39,5.09,13.56,1.13,2,43,39,OF,-1,-1
3,3,Li,Li_OF_052818,OF,052818,0000,8,16.25,Li_OF_052818-tt8_d16.25_cl4,4,...,44,10.08,2.95,0.44,3,48,44,OF,-1,-1
4,4,Li,Li_OF_052818,OF,052818,0000,8,16.25,Li_OF_052818-tt8_d16.25_cl18,18,...,52,6.6,33.03,0.54,4,50,52,OF,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,73,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl20,20,...,8,8.11,25.24,0.11,4459,964,858,OF,-1,-1
4460,74,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl10,10,...,NaN,6.75,8.17,0.51,4460,NaN,NaN,OF,-1,-1
4461,75,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl1,1,...,11,12.38,4.92,0,4461,967,861,OF,-1,-1
4462,76,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl0,0,...,NaN,20.18,0.42,0,4462,NaN,NaN,OF,-1,-1


0.9198250728862973

# Dev and Debug from here

## Update Clusters if changes in phy

In [226]:
%%time
si = reload(si)
subjects = ['Li', 'Ne', 'Cl', 'Al', 'Ca', 'Mi']
overwrite = False
for subject in subjects:
    subject_info = si.SubjectInfo(subject, overwrite=overwrite)
    subject_info.update_clusters()   ## updates cluster snr/ fr isi info based on phy changes.

Error Computing Cluster Stats for Li_OF_052818
CPU times: user 3min 51s, sys: 43.7 s, total: 4min 35s
Wall time: 4min 44s


## Update cluster matches

In [227]:
%%time
si = reload(si)
subjects = ['Ne', 'Li', 'Cl', 'Al', 'Ca', 'Mi']
overwrite = True
for subject in subjects:
    try:
        subject_info = si.SubjectInfo(subject, overwrite=overwrite)
        subject_info.get_cluster_dists(overwrite=overwrite)
        subject_info.match_clusters(overwrite=overwrite)
        subject_info.match_clusters(overwrite=overwrite, require_subsets=False)
    except:
        print(f"{subject} failed.")

.....................

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


.

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


..........Error Computing Cluster Stats for Li_OF_052818


...

.....

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


....................

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


............CPU times: user 4h 52min 32s, sys: 20min 40s, total: 5h 13min 13s
Wall time: 47min 26s


In [2]:
subjects = ['Ne', 'Li', 'Cl', 'Al', 'Ca', 'Mi']

In [4]:
subject = 'Cl'
session_num = 10

subject_info = si.SubjectInfo(subject)
session = subject_info.sessions[session_num]
subject_info.session_clusters[session].keys()

dict_keys(['session', 'path', 'n_cell', 'n_mua', 'n_noise', 'n_unsorted', 'sorted_TTs', 'curated_TTs', 'cell_IDs', 'mua_IDs', 'noise_IDs', 'unsorted_IDs', 'clusters_snr', 'clusters_fr', 'clusters_valid', 'clusters_isi_viol_rate'])

In [156]:
si = reload(si)
subject_info = si.SubjectInfo(subject)
subject_info.get_session_match_analysis()
subject_info.get_cluster_dists(overwrite=True)


............

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


.......

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


...

...

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


....

/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/cluster_match_functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  h = np.sqrt(h2)


.....

{0: {'analysis': {'tt': 1,
   'd': 12.0,
   'n_units': 2,
   'sessions': ['Cl_T3g_112118_0001', 'Cl_OF_112118'],
   'n_session_units': [1, 1]},
  'cl_names': ['Cl_T3g_112118_0001-tt1_d12.0_cl10',
   'Cl_OF_112118-tt1_d12.0_cl14'],
  'clusters_loc': {'Cl_T3g_112118_0001-tt1_d12.0_cl10': array([-5.9317174,  3.5179584]),
   'Cl_OF_112118-tt1_d12.0_cl14': array([12.24919224,  3.15104485])},
  'clusters_cov': {'Cl_T3g_112118_0001-tt1_d12.0_cl10': array([[0.75316474, 0.01697233],
          [0.01697233, 0.89957466]]),
   'Cl_OF_112118-tt1_d12.0_cl14': array([[0.70926588, 0.22480795],
          [0.22480795, 1.06707396]])},
  'dists_mats': {'he':                                    Cl_T3g_112118_0001-tt1_d12.0_cl10  \
   Cl_T3g_112118_0001-tt1_d12.0_cl10                                0.0   
   Cl_OF_112118-tt1_d12.0_cl14                                      1.0   
   
                                      Cl_OF_112118-tt1_d12.0_cl14  
   Cl_T3g_112118_0001-tt1_d12.0_cl10                        

In [160]:
overwrite=True
si = reload(si)
subject_info = si.SubjectInfo(subject)
subject_info.match_clusters(overwrite=overwrite)
subject_info.match_clusters(overwrite=overwrite, require_subsets=False)

{0: {'analysis': {'tt': 1,
   'd': 12.0,
   'n_units': 2,
   'sessions': ['Cl_T3g_112118_0001', 'Cl_OF_112118'],
   'n_session_units': [1, 1]},
  'matches_dict': {'Cl_T3g_112118_0001-tt1_d12.0_cl10': [set()],
   'Cl_OF_112118-tt1_d12.0_cl14': [set()]},
  'matches_sets': [{'Cl_T3g_112118_0001-tt1_d12.0_cl10'},
   {'Cl_OF_112118-tt1_d12.0_cl14'}]},
 1: {'analysis': {'tt': 1,
   'd': 12.25,
   'n_units': 2,
   'sessions': ['Cl_T3g_112318', 'Cl_T3g_112818'],
   'n_session_units': [1, 1]},
  'matches_dict': {'Cl_T3g_112318-tt1_d12.25_cl13': [set()],
   'Cl_T3g_112818-tt1_d12.25_cl9': [set()]},
  'matches_sets': [{'Cl_T3g_112318-tt1_d12.25_cl13'},
   {'Cl_T3g_112818-tt1_d12.25_cl9'}]},
 2: {'analysis': {'tt': 2,
   'd': 12.0,
   'n_units': 7,
   'sessions': ['Cl_T3g_112118_0001', 'Cl_OF_112118'],
   'n_session_units': [3, 4]},
  'matches_dict': {'Cl_T3g_112118_0001-tt2_d12.0_cl19': [set()],
   'Cl_OF_112118-tt2_d12.0_cl18': [{'Cl_T3g_112118_0001-tt2_d12.0_cl23'}],
   'Cl_T3g_112118_0001-tt2_

In [154]:
matching_analyses = subject_info.get_session_match_analysis()
for ii, a in matching_analyses.items():
    print(a)
    break


{'tt': 1, 'd': 12.0, 'n_units': 2, 'sessions': ['Cl_T3g_112118_0001', 'Cl_OF_112118'], 'n_session_units': [1, 1]}


In [161]:
n_total_units = 0
for session in subject_info.sessions:    
    n_total_units+=subject_info.session_clusters[session]['n_cell']
    n_total_units+=subject_info.session_clusters[session]['n_mua']

subject_clusters_table = pd.DataFrame(index=np.arange(n_total_units), columns=["subject_cl_id", "subject", "session", "task", "date", "subsession", "tt", "depth", "unique_cl_name",
                                                                              "session_cl_id", "unit_type", "n_matches_con", "subject_cl_match_con_id","n_matches_lib" ,"subject_cl_match_lib_id", 
                                                                              "snr", "fr", "isi_viol_rate"])
subject_clusters_table["subject"] = subject

subject_cl_matches_con = subject_info.match_clusters()
matches_con_sets = {}
matches_con_set_num = {}
matches_con_dict = {}
cnt = 0
for k, cma in subject_cl_matches_con.items():
    matches_con_sets.update({cnt+ii: clx_set for ii, clx_set in enumerate(cma['matches_sets'])})
    cnt = len(matches_con_sets)
    matches_con_dict.update(cma['matches_dict'])
    
for set_num, clm_set in matches_con_sets.items():
    for cl in clm_set:
        matches_con_set_num[cl] = set_num
        
subject_cl_matches_lib = subject_info.match_clusters(require_subsets=False)
matches_lib_sets = {}
matches_lib_set_num = {} 
matches_lib_dict = {}
cnt = 0
for k, cma in subject_cl_matches_lib.items():
    matches_lib_sets.update({cnt+ii: clx_set for ii, clx_set in enumerate(cma['matches_sets'])})
    cnt = len(matches_lib_sets)
    matches_lib_dict.update(cma['matches_dict'])
    
for set_num, clm_set in matches_lib_sets.items():
    for cl in clm_set:
        matches_lib_set_num[cl] = set_num
        
unit_cnt = 0
for session in subject_info.sessions:
    session_details = session.split("_")

    if len(session_details)>3:
        subsession = session_details[3]
    else:
        subsession = "0000"

    session_clusters = subject_info.session_clusters[session]

    n_session_cells = session_clusters['n_cell']
    n_session_mua = session_clusters['n_mua']
    n_session_units = n_session_cells + n_session_mua

    session_unit_idx = np.arange(n_session_units)+unit_cnt

    subject_clusters_table.loc[session_unit_idx, "subject_cl_id"] = session_unit_idx
    subject_clusters_table.loc[session_unit_idx, "session"] = session
    subject_clusters_table.loc[session_unit_idx, "task"] = session_details[1]
    subject_clusters_table.loc[session_unit_idx, "date"] = session_details[2]
    subject_clusters_table.loc[session_unit_idx, "subsession"] = subsession

    cell_IDs = session_clusters['cell_IDs']
    mua_IDs = session_clusters['mua_IDs']

    for unit_type in ['cell', 'mua']:
        for tt, tt_clusters in session_clusters[f'{unit_type}_IDs'].items():
            if len(tt_clusters)>0:
                depth = subject_info.sessions_tt_positions.loc[session, f"tt_{tt}"]
                for cl in tt_clusters:
                    cl_name = f"{session}-tt{tt}_d{depth}_cl{cl}"
                    subject_clusters_table.loc[unit_cnt, "subject_cl_id"] = unit_cnt
                    subject_clusters_table.loc[unit_cnt, "unique_cl_name"] = cl_name
                    subject_clusters_table.loc[unit_cnt, "tt"] = tt
                    subject_clusters_table.loc[unit_cnt, "depth"] = depth
                    subject_clusters_table.loc[unit_cnt, "unit_type"] = unit_type
                    subject_clusters_table.loc[unit_cnt, "session_cl_id"] = cl

                    subject_clusters_table.loc[unit_cnt, "snr" ]= session_clusters["clusters_snr"][tt][cl]
                    subject_clusters_table.loc[unit_cnt, "fr" ]= session_clusters["clusters_fr"][tt][cl]
                    subject_clusters_table.loc[unit_cnt, "isi_viol_rate" ]= session_clusters["clusters_isi_viol_rate"][tt][cl]

                    if unit_type == 'cell':

                        # add fields of conservative cluster matching (requires subset)
                        if cl_name in matches_con_dict.keys():
                            cl_matches = matches_con_dict[cl_name][0]
                            subject_clusters_table.loc[unit_cnt, "n_matches_con"] = len(cl_matches)
                            subject_clusters_table.loc[unit_cnt, "subject_cl_match_con_id"] = matches_con_set_num[cl_name]

                        # add fields of liberal cluster matching ( does not require subset matching)
                        if cl_name in matches_lib_dict.keys():
                            cl_matches = matches_lib_dict[cl_name][0]
                            subject_clusters_table.loc[unit_cnt, "n_matches_lib"] = len(cl_matches)
                            subject_clusters_table.loc[unit_cnt, "subject_cl_match_lib_id"] = matches_lib_set_num[cl_name]

                    unit_cnt+=1
    

In [173]:
subject_clusters_table

,subject_cl_id,subject,session,task,date,subsession,tt,depth,unique_cl_name,session_cl_id,unit_type,n_matches_con,subject_cl_match_con_id,n_matches_lib,subject_cl_match_lib_id,snr,fr,isi_viol_rate
0,0,Cl,Cl_T3g_010218,T3g,010218,0000,3,14.5,Cl_T3g_010218-tt3_d14.5_cl23,23,cell,1,54,1,54,14.59,36.32,0.34
1,1,Cl,Cl_T3g_010218,T3g,010218,0000,4,14.25,Cl_T3g_010218-tt4_d14.25_cl0,0,cell,0,64,0,64,14.5,34.2,0
2,2,Cl,Cl_T3g_010218,T3g,010218,0000,4,14.25,Cl_T3g_010218-tt4_d14.25_cl3,3,cell,0,65,0,65,6.58,1.23,0.32
3,3,Cl,Cl_T3g_010218,T3g,010218,0000,4,14.25,Cl_T3g_010218-tt4_d14.25_cl16,16,cell,1,67,2,72,13.77,16.19,0.05
4,4,Cl,Cl_T3g_010218,T3g,010218,0000,10,13.75,Cl_T3g_010218-tt10_d13.75_cl6,6,cell,1,279,2,245,9.65,7.11,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,2001,Cl,Cl_T3g_021719,T3g,021719,0000,4,16.75,Cl_T3g_021719-tt4_d16.75_cl12,12,mua,NaN,NaN,NaN,NaN,7.21,1.16,3.86
2002,2002,Cl,Cl_T3g_021719,T3g,021719,0000,4,16.75,Cl_T3g_021719-tt4_d16.75_cl15,15,mua,NaN,NaN,NaN,NaN,6.18,1.13,18.04
2003,2003,Cl,Cl_T3g_021719,T3g,021719,0000,10,16.5,Cl_T3g_021719-tt10_d16.5_cl11,11,mua,NaN,NaN,NaN,NaN,5.87,2.8,0.71
2004,2004,Cl,Cl_T3g_021719,T3g,021719,0000,10,16.5,Cl_T3g_021719-tt10_d16.5_cl12,12,mua,NaN,NaN,NaN,NaN,6.46,2.61,4.02


## Debug the implementation:

In [184]:
si = reload(si)
subject = 'Cl'
subject_info = si.SubjectInfo(subject)
tbl = subject_info.get_units_table()


In [194]:
tbl

,subject_cl_id,subject,session,task,date,subsession,tt,depth,unique_cl_name,session_cl_id,unit_type,n_matches_con,subject_cl_match_con_id,n_matches_lib,subject_cl_match_lib_id,snr,fr,isi_viol_rate
0,0,Cl,Cl_T3g_010218,T3g,010218,0000,3,14.5,Cl_T3g_010218-tt3_d14.5_cl23,23,cell,1,54,1,54,14.59,36.32,0.34
1,1,Cl,Cl_T3g_010218,T3g,010218,0000,4,14.25,Cl_T3g_010218-tt4_d14.25_cl0,0,cell,0,64,0,64,14.5,34.2,0
2,2,Cl,Cl_T3g_010218,T3g,010218,0000,4,14.25,Cl_T3g_010218-tt4_d14.25_cl3,3,cell,0,65,0,65,6.58,1.23,0.32
3,3,Cl,Cl_T3g_010218,T3g,010218,0000,4,14.25,Cl_T3g_010218-tt4_d14.25_cl16,16,cell,1,67,2,72,13.77,16.19,0.05
4,4,Cl,Cl_T3g_010218,T3g,010218,0000,10,13.75,Cl_T3g_010218-tt10_d13.75_cl6,6,cell,1,279,2,245,9.65,7.11,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,2001,Cl,Cl_T3g_021719,T3g,021719,0000,4,16.75,Cl_T3g_021719-tt4_d16.75_cl12,12,mua,NaN,NaN,NaN,NaN,7.21,1.16,3.86
2002,2002,Cl,Cl_T3g_021719,T3g,021719,0000,4,16.75,Cl_T3g_021719-tt4_d16.75_cl15,15,mua,NaN,NaN,NaN,NaN,6.18,1.13,18.04
2003,2003,Cl,Cl_T3g_021719,T3g,021719,0000,10,16.5,Cl_T3g_021719-tt10_d16.5_cl11,11,mua,NaN,NaN,NaN,NaN,5.87,2.8,0.71
2004,2004,Cl,Cl_T3g_021719,T3g,021719,0000,10,16.5,Cl_T3g_021719-tt10_d16.5_cl12,12,mua,NaN,NaN,NaN,NaN,6.46,2.61,4.02


In [350]:
si = reload(si)

unit_cnt=0
unit_con_match_cnt = 0
unit_lib_match_cnt = 0

all_units_table = pd.DataFrame()
for subject in subjects:
    subject_info = si.SubjectInfo(subject)
    
    tbl = subject_info.get_units_table(overwrite=True)
    tbl['cl_id'] = tbl.subject_cl_id + unit_cnt 
    tbl['cl_match_con_id'] =  tbl.subject_cl_match_con_id + unit_con_match_cnt 
    tbl['cl_match_lib_id'] =  tbl.subject_cl_match_lib_id + unit_lib_match_cnt
    
    unit_cnt = np.nanmax(tbl.cl_id)+1
    unit_con_match_cnt = np.nanmax(tbl.cl_match_con_id)+1
    unit_lib_match_cnt = np.nanmax(tbl.cl_match_lib_id)+1
    
    all_units_table = pd.concat((all_units_table, tbl))
all_units_table =  all_units_table.reset_index(drop=True)

def isT3(a):
    return 'T3' in a

all_units_table['task2'] = all_units_table.task
all_units_table.loc[list(map(isT3, all_units_table.task)),'task2'] = 'T3'


In [351]:
all_units_table

,subject_cl_id,subject,session,task,date,subsession,tt,depth,unique_cl_name,session_cl_id,...,subject_cl_match_con_id,n_matches_lib,subject_cl_match_lib_id,snr,fr,isi_viol_rate,cl_id,cl_match_con_id,cl_match_lib_id,task2
0,0,Li,Li_T3g_052818,T3g,052818,0000,2,16.5,Li_T3g_052818-tt2_d16.5_cl11,11,...,NaN,NaN,NaN,NaN,2.84,0.2,0,NaN,NaN,T3
1,1,Li,Li_T3g_052818,T3g,052818,0000,2,16.5,Li_T3g_052818-tt2_d16.5_cl14,14,...,NaN,NaN,NaN,NaN,2.69,0.05,1,NaN,NaN,T3
2,2,Li,Li_OF_052818,OF,052818,0000,7,17.25,Li_OF_052818-tt7_d17.25_cl8,8,...,43,0,39,5.09,13.56,1.13,2,43,39,OF
3,3,Li,Li_OF_052818,OF,052818,0000,8,16.25,Li_OF_052818-tt8_d16.25_cl4,4,...,48,1,44,10.08,2.95,0.44,3,48,44,OF
4,4,Li,Li_OF_052818,OF,052818,0000,8,16.25,Li_OF_052818-tt8_d16.25_cl18,18,...,50,2,52,6.6,33.03,0.54,4,50,52,OF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,73,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl20,20,...,8,0,8,8.11,25.24,0.11,4459,964,858,OF
4460,74,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl10,10,...,NaN,NaN,NaN,6.75,8.17,0.51,4460,NaN,NaN,OF
4461,75,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl1,1,...,11,0,11,12.38,4.92,0,4461,967,861,OF
4462,76,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl0,0,...,NaN,NaN,NaN,20.18,0.42,0,4462,NaN,NaN,OF


In [ ]:
subject_info.match_clusters(overwrite=overwrite)
cm = subject_info.match_clusters(overwrite=overwrite, require_subsets=False)

In [284]:
cm = subject_info.match_clusters(overwrite=overwrite, require_subsets=False)

In [310]:
lib_matches_sets = []
for subject in subjects:
    subject_info = si.SubjectInfo(subject)
    cm = subject_info.match_clusters(overwrite=overwrite, require_subsets=False)
    for k, cma in cm.items():
        lib_matches_sets+=cma['matches_sets']

In [384]:
all_units_table['match_lib_multi_task_id'] = -1
unit_matches = np.arange(np.nanmax(all_units_table.cl_match_lib_id)+1)
cnt=0
for cm  in unit_matches:
    tasks_in_cm = all_units_table[all_units_table.cl_match_lib_id==cm].task2.values
    if ('OF' in tasks_in_cm) & ('T3' in tasks_in_cm):
        all_units_table.loc[all_units_table.cl_match_lib_id==cm, 'match_lib_multi_task_id'] = cnt
        cnt+=1
                                  
all_units_table['match_con_multi_task_id'] = -1
unit_matches = np.arange(np.nanmax(all_units_table.cl_match_con_id)+1)
cnt=0
for cm  in unit_matches:
    tasks_in_cm = all_units_table[all_units_table.cl_match_con_id==cm].task2.values
    if ('OF' in tasks_in_cm) & ('T3' in tasks_in_cm):
        all_units_table.loc[all_units_table.cl_match_con_id==cm, 'match_con_multi_task_id'] = cnt
        cnt+=1

In [386]:
np.max(all_units_table.match_lib_multi_task_id)

214

In [374]:
len(all_units_table[all_units_table.match_con_multi_task].cl_match_con_id.unique()), len(all_units_table[all_units_table.match_lib_multi_task].cl_match_lib_id.unique())

(194, 215)

In [388]:
all_units_table

,subject_cl_id,subject,session,task,date,subsession,tt,depth,unique_cl_name,session_cl_id,...,subject_cl_match_lib_id,snr,fr,isi_viol_rate,cl_id,cl_match_con_id,cl_match_lib_id,task2,match_lib_multi_task_id,match_con_multi_task_id
0,0,Li,Li_T3g_052818,T3g,052818,0000,2,16.5,Li_T3g_052818-tt2_d16.5_cl11,11,...,NaN,NaN,2.84,0.2,0,NaN,NaN,T3,-1,-1
1,1,Li,Li_T3g_052818,T3g,052818,0000,2,16.5,Li_T3g_052818-tt2_d16.5_cl14,14,...,NaN,NaN,2.69,0.05,1,NaN,NaN,T3,-1,-1
2,2,Li,Li_OF_052818,OF,052818,0000,7,17.25,Li_OF_052818-tt7_d17.25_cl8,8,...,39,5.09,13.56,1.13,2,43,39,OF,-1,-1
3,3,Li,Li_OF_052818,OF,052818,0000,8,16.25,Li_OF_052818-tt8_d16.25_cl4,4,...,44,10.08,2.95,0.44,3,48,44,OF,-1,-1
4,4,Li,Li_OF_052818,OF,052818,0000,8,16.25,Li_OF_052818-tt8_d16.25_cl18,18,...,52,6.6,33.03,0.54,4,50,52,OF,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,73,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl20,20,...,8,8.11,25.24,0.11,4459,964,858,OF,-1,-1
4460,74,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl10,10,...,NaN,6.75,8.17,0.51,4460,NaN,NaN,OF,-1,-1
4461,75,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl1,1,...,11,12.38,4.92,0,4461,967,861,OF,-1,-1
4462,76,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl0,0,...,NaN,20.18,0.42,0,4462,NaN,NaN,OF,-1,-1


In [390]:
all_units_table[all_units_table.match_con_multi_task_id>=0].subject.value_counts()

Cl    214
Li    112
Ne     50
Ca      8
Al      2
Mi      2
Name: subject, dtype: int64

In [395]:
session_info = si.SubjectSessionInfo('Mi', 'Mi_OF_021820')

In [398]:
session_info.cluster_ids['0']

['cell', '14', '11']

In [401]:
subject_info.session_clusters['Mi_OF_021820']

{'session': 'Mi_OF_021820',
 'path': '/mnt/Data3_SSD2T/Data/Sorted/Mi/Mi_OF_021820',
 'n_cell': 1,
 'n_mua': 0,
 'n_noise': 14,
 'n_unsorted': 230,
 'sorted_TTs': [1, 2, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16],
 'curated_TTs': [14],
 'cell_IDs': {1: [],
  2: [],
  3: [],
  4: [],
  6: [],
  7: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: [11],
  15: [],
  16: []},
 'mua_IDs': {1: [],
  2: [],
  3: [],
  4: [],
  6: [],
  7: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: [],
  15: [],
  16: []},
 'noise_IDs': {1: [],
  2: [],
  3: [],
  4: [],
  6: [],
  7: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14],
  15: [],
  16: []},
 'unsorted_IDs': {1: [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18],
  2: [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  3: [0, 1, 2, 3, 4, 5, 6,

In [331]:
def isT3(a):
    return 'T3' in a

all_units_table['task2'] = all_units_table.task
all_units_table.loc[list(map(isT3, all_units_table.task))] = 'T3'
all_units_table
#set(tasks_in_sets).intersection(})

,subject_cl_id,subject,session,task,date,subsession,tt,depth,unique_cl_name,session_cl_id,...,subject_cl_match_con_id,n_matches_lib,subject_cl_match_lib_id,snr,fr,isi_viol_rate,cl_id,cl_match_con_id,cl_match_lib_id,task2
0,0,Ne,NE_OF_052818,OF,052818,0000,2,13,NE_OF_052818-tt2_d13.0_cl6,6,...,NaN,NaN,NaN,5.98,16.12,0.05,0,NaN,NaN,OF
1,1,Ne,NE_OF_052818,OF,052818,0000,2,13,NE_OF_052818-tt2_d13.0_cl10,10,...,NaN,NaN,NaN,7.22,6.95,0.05,1,NaN,NaN,OF
2,2,Ne,NE_OF_052818,OF,052818,0000,2,13,NE_OF_052818-tt2_d13.0_cl12,12,...,NaN,NaN,NaN,4.18,9.87,0.06,2,NaN,NaN,OF
3,3,Ne,NE_OF_052818,OF,052818,0000,2,13,NE_OF_052818-tt2_d13.0_cl13,13,...,NaN,NaN,NaN,4.7,3.23,0.12,3,NaN,NaN,OF
4,4,Ne,NE_OF_052818,OF,052818,0000,2,13,NE_OF_052818-tt2_d13.0_cl14,14,...,NaN,NaN,NaN,4.01,5.05,0.15,4,NaN,NaN,OF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,73,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl20,20,...,8,0,8,8.11,25.24,0.11,4459,964,858,OF
4460,74,Mi,Mi_OF_021220,OF,021220,0000,12,16.125,Mi_OF_021220-tt12_d16.125_cl10,10,...,NaN,NaN,NaN,6.75,8.17,0.51,4460,NaN,NaN,OF
4461,75,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl1,1,...,11,0,11,12.38,4.92,0,4461,967,861,OF
4462,76,Mi,Mi_OF_021720,OF,021720,0000,14,16.375,Mi_OF_021720-tt14_d16.375_cl0,0,...,NaN,NaN,NaN,20.18,0.42,0,4462,NaN,NaN,OF


array(['OF', 'T3'], dtype=object)

In [262]:
np.nanmax(tbl.subject_cl_match_con_id), np.nanmax(tbl.subject_cl_match_lib_id) 

(29, 28)

In [218]:
subject_info.session_clusters["Li_OF_052818"]["clusters_snr"]

{2: {0: 3.53,
  1: 4.98,
  2: 3.67,
  3: 1.37,
  4: 3.75,
  5: 5.11,
  6: 6.55,
  7: 2.84,
  8: 6.45,
  9: 2.25,
  10: 1.35,
  11: 1.4,
  12: 1.9,
  13: 11.14,
  14: 1.54,
  15: 4.75,
  16: 7.13,
  17: 4.3},
 4: {0: 3.21,
  1: 6.83,
  2: 1.88,
  3: 1.59,
  4: 1.59,
  5: 1.47,
  6: 1.43,
  7: 14.28,
  8: 8.96,
  9: 3.14,
  10: 21.3,
  11: 1.06,
  12: 0.91,
  13: 1.41,
  14: 5.3,
  15: 1.56,
  16: 3.36,
  17: 1.28},
 5: {0: nan,
  1: nan,
  2: nan,
  3: nan,
  4: nan,
  5: nan,
  6: nan,
  7: nan,
  8: nan,
  9: nan,
  10: nan,
  11: nan,
  12: nan,
  13: nan,
  14: nan,
  15: nan,
  16: nan},
 6: {0: 4.87,
  1: 5.4,
  2: 1.95,
  3: 10.65,
  4: 3.24,
  5: 4.12,
  6: 6.6,
  7: 7.12,
  8: 7.29,
  9: 8.74,
  10: 6.73,
  11: 4.41,
  12: 1.26,
  13: 4.27,
  14: 2.04,
  15: 4.52,
  16: 16.53,
  17: 9.86,
  18: 4.6,
  19: 7.42,
  20: 2.64,
  21: 5.54},
 7: {0: 2.77,
  1: 2.74,
  2: 4.34,
  3: 3.96,
  4: 6.43,
  5: 0.98,
  6: 2.08,
  7: 5.3,
  8: 5.09,
  9: 4.25,
  10: 2.79,
  11: 0.69,
  12: 1.

In [220]:
subject_info.session_clusters["Li_OF_052818"]["cell_IDs"]

{2: [], 4: [], 5: [], 6: [], 7: [8], 8: [4, 18], 11: [], 13: []}

In [225]:
si = reload(si)
subject_info = si.SubjectInfo(subject)
subject_info._cluster_stats("Li_OF_052818")

Traceback (most recent call last):
  File "/home/alexgonzalez/Documents/TreeMazeAnalyses2/Analyses/subject_info.py", line 646, in _cluster_stats
    cluster_groups = pd.read_csv(_cluster_groups_fn, sep='\t')
  File "/home/alexgonzalez/anaconda3/envs/spk_phy2/lib/python3.7/site-packages/pandas/io/parsers.py", line 688, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/alexgonzalez/anaconda3/envs/spk_phy2/lib/python3.7/site-packages/pandas/io/parsers.py", line 454, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/home/alexgonzalez/anaconda3/envs/spk_phy2/lib/python3.7/site-packages/pandas/io/parsers.py", line 948, in __init__
    self._make_engine(self.engine)
  File "/home/alexgonzalez/anaconda3/envs/spk_phy2/lib/python3.7/site-packages/pandas/io/parsers.py", line 1180, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/home/alexgonzalez/anaconda3/envs/spk_phy2/lib/python3.7/site-packages/pandas/io/parsers.py", line 2010